<style>
    .title_div {
    padding: 60px;
    text-align: center;
    background: #ffffff;
    }
    #title {
        color: #f2a900;
        font-size: 80px;
    }
    #subtitle {
        color: #4d4d4e;
        font-size: 20px;
    }
</style>

<div class="title_div">
  <h1 id="title">TX</h1>
  <p id="subtitle">Transactions and Script</p>
</div>

<div style="color:red; text-align: center; padding: 60px">
⚠ The following TX examples are just to see how Bitcoin work; don't transmit nothing generated here on the mainnet. ⚠
</div>


## TOC:
* [Hashes](#hasehs)
    * [Hash256](#hash256)
    * [Hash160](#hash160)
* [VarInt](#varint)
* [TxIn](#txin)
* [TxOut](#txout)
* [TX](#tx)
* [Script](#script)
    * [P2PK](#p2pk)
    * [P2PKH](#p2pkh)
    * [P2SH](#p2sh)

In [204]:
from io import StringIO, BytesIO
from hashlib import sha256, new
import unittest
import base58

<a class="anchor" id="hashes"></a>
## Hashes

<a class="anchor" id="hash256"></a>
### Hash256

In [205]:
def hash256(data):
    return sha256(sha256(data).digest()).digest()

<a class="anchor" id="hash160"></a>
### Hash160

In [206]:
def hash160(data):
    return new('ripemd160', sha256(data).digest()).digest()

<a class="anchor" id="varint"></a>
## VarInt

In [207]:
def read_varint(s):
    
    flag = s.read(1)
    if flag == 0xfd:
        # 0xfd next two bytes are the number
        return flag.read(2)
    elif flag == 0xfe:
        # 0xfe next four bytes are the number
        return flag.read(4)
    elif flag == 0xff:
        # 0xff next eight bytes are the number
        return flag.read(8)
    else:
        # the flag is the number
        return flag


def encode_varint(i):
   
    if i < 0xfd:
        return bytes([i])
    elif i < 0x10000:
        return b'\xfd' + i.to_bytes(2, 'litte')
    elif i < 0x100000000:
        return b'\xfe' + i.to_bytes(4, 'litte')
    elif i < 0x10000000000000000:
        return b'\xff' + i.to_bytes(8, 'litte')
    else:
        raise ValueError('integer too large: {}'.format(i))

<a class="anchor" id="txin"></a>
## TxIn

In [208]:
class TxIn:
    def __init__(self, txID, outID, scriptSig = None, sequence = 0xffffffff):
        self.txID = txID # hex string
        self.outIndex = outID # int
        self.scriptSig = scriptSig # 
        self.sequence = sequence
    
    def serialize(self):
        serialized_tx = self.txID[::-1]
        serialized_tx += self.outIndex[::-1]
        if self.scriptSig != None:
            serialized_tx += encode_varint(len(self.scriptSig))
            serialized_tx += self.scriptSig
        serialized_tx += self.sequence[::-1]

        return serialized_tx


    @classmethod
    def parse(cls, input, hex=False):
        if hex:
            stream = input
        else:
            stream = BytesIO(bytes.fromhex(input))
        TxID = stream.read(32)[::-1]
        index = stream.read(4)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        secuence = stream.read(4)[::-1]
        return cls(TxID, index, script, secuence)

    def __repr__(self):
        string = '[\n\t'
        string += f'TxID: {self.txID.hex()}\n\t'
        string += f'OutIndex: {self.outIndex.hex()}\n\t'
        if self.scriptSig != None:
            string += f'Script size: {len(self.scriptSig)}\n\t'
            string += f'Script: {self.scriptSig.hex()}\n\t'
        string += f'Sequence: {self.sequence.hex()}\n' + ']'
        return string

    def colors(self):
        result = ''
        result += '\033[94m' + self.txID[::-1].hex()
        result += '\033[91m' + self.outIndex[::-1].hex()
        if self.scriptSig != None:
            result += '\033[93m' + encode_varint(len(self.scriptSig)).hex()
            result += '\033[96m' + self.scriptSig.hex()
        result += '\033[95m' + self.sequence[::-1].hex()
        result += '\n\033[94m\u2589 TxID'
        result += '\n\033[91m\u2589 Index'
        result += '\n\033[93m\u2589 Script Size'
        result += '\n\033[96m\u2589 ScriptSig'
        result += '\n\033[95m\u2589 Sequence'

        return result

In [209]:
tx_in='7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc01000000\
    6a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f227\
    02201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b\
    7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff'

print(TxIn.parse(tx_in).colors())

7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff
▉ TxID
▉ Index
▉ Script Size
▉ ScriptSig
▉ Sequence


<a class="anchor" id="txout"></a>
## TxOut

In [210]:
class TxOut:

    def __init__(self, value, scriptPubKey):
        self.value = value
        self.scriptPubKey = scriptPubKey

    def serialize(self):
        serialized_tx = self.value[::-1]
        serialized_tx += encode_varint(len(self.scriptPubKey))
        serialized_tx += self.scriptPubKey
        
        return serialized_tx

    @classmethod
    def parse(cls, output, hex=False):
        if hex:
            stream = output
        else:
            stream = BytesIO(bytes.fromhex(output))
        value = stream.read(8)[::-1]
        script_size = int.from_bytes(read_varint(stream), 'little')
        script = stream.read(script_size)
        return cls(value, script)

    def __repr__(self):
        string = '[\n\t'
        string += f'Value: {int.from_byes(self.value, "big")}\n\t'
        string += f'Script size: {len(self.scriptPubKey)}\n\t'
        string += f'Script: {self.scriptPubKey.hex()}\n' + ']'
        return string

    def colors(self):
        result = ''
        result += '\033[91m' + self.value[::-1].hex()
        result += '\033[93m' + encode_varint(len(self.scriptPubKey)).hex()
        result += '\033[96m' + self.scriptPubKey.hex()
        result += '\n\033[91m\u2589 Value'
        result += '\n\033[93m\u2589 Script Size'
        result += '\n\033[96m\u2589 ScriptPubKey'
        return result


In [211]:
tx_out='4baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac'

print(TxOut.parse(tx_out).colors())

4baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac
▉ Value
▉ Script Size
▉ ScriptPubKey


<a class="anchor" id="tx"></a>
## TX

In [212]:
class TX:
    def __init__(self, inputs, outputs, version=0x00000001 , locktime=0x00000000):
        self.inputs = inputs
        self.outputs = outputs
        self.version = version
        self.locktime = locktime

    def serialize(self):
        serialized_tx = self.version[::-1]
        serialized_tx += encode_varint(len(self.inputs))
        for i in self.inputs:
            serialized_tx += i.serialize()
        
        serialized_tx += encode_varint(len(self.outputs))
        for i in self.outputs:
            serialized_tx += i.serialize()

        serialized_tx += self.locktime[::-1]
        return serialized_tx

    @classmethod
    def parse(cls, tx):
        stream = BytesIO(bytes.fromhex(tx))

        version = stream.read(4)[::-1]

        input_count = int.from_bytes(read_varint(stream), 'little')
        inputs = []
        for i in range(input_count):
            inputs.append(TxIn.parse(stream, True))

        output_count = int.from_bytes(read_varint(stream), 'little')
        outputs = []
        for i in range(output_count):
            outputs.append(TxOut.parse(stream, True))
        locktime = stream.read(4)[::-1]

        return cls(inputs, outputs, version, locktime)

    def colors(self):
        result = ''
        result += '\033[94m' + self.version[::-1].hex()

        result += '\033[93m' + encode_varint(len(self.inputs)).hex()
        result += '\033[96m'
        for i in self.inputs:
            result += i.serialize().hex()
        
        result += '\033[93m' + encode_varint(len(self.outputs)).hex()
        result += '\033[95m'
        for i in self.outputs:
            result += i.serialize().hex()

        result += '\033[92m' + self.locktime[::-1].hex()

        result += '\n\033[94m\u2589 Version'
        result += '\n\033[93m\u2589 Counter'
        result += '\n\033[96m\u2589 Inputs'
        result += '\n\033[95m\u2589 Outputs'
        result += '\n\033[92m\u2589 Sequence'

        return result

In [213]:
transaction = TX.parse('01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc\
    010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b\
    10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63\
    e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf2100000000001976a914db4d1141d0048b1ed15839d0b7\
    a4c488cd368b0e88ac00000000')
print(transaction.colors())

01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac00000000
▉ Version
▉ Counter
▉ Inputs
▉ Outputs
▉ Sequence


<a class="anchor" id="script"></a>
## Script

In [246]:
def data_encoding(data):
    count = len(data)
    if count < 76:
        return count.to_bytes(1, 'little') + data
    elif count < 2**8:
       return b'\x4a' + count.to_bytes(1, 'little') + data #OP_PUSHDATA1
    elif count < 2**16: # cap at 520
        return b'\x4d' + count.to_bytes(2, 'little') + data #OP_PUSHDATA2
    elif count < 2**32:
        return b'\x4e' + count.to_bytes(4, 'little') + data #OP_PUSHDATA4

def data_decoding(data):
    code = int.from_bytes(data.read(1), 'little')
    if code == 0x4a:
        size = int.from_bytes(data.read(1), 'little') #OP_PUSHDATA1
        return data.read(size)
    elif code == 0x4d:
        size = int.from_bytes(data.read(2), 'little') #OP_PUSHDATA2
        return data.read(size)
    elif code == 0x4e:
        size = int.from_bytes(data.read(4), 'little') #OP_PUSHDATA4
        return data.read(size)
    else:
        return data.read(code)

<a class="anchor" id="p2pk"></a>
### P2PK

In [215]:
len('30440220576497b7e6f9b553c0aba0d8929432550e092db9c130aae37b84b545e7f4a36c022066cb982ed80608372c139d7bb9af335423d5280350fe3e06bd510e695480914f01')

142

In [216]:
int.from_bytes(bytes.fromhex('47'), 'big')

71

<a class="anchor" id="p2pkh"></a>
### P2PKH

<a class="anchor" id="p2sh"></a>
### P2SH

In [238]:
class TestTX(unittest.TestCase):
    def test_TxIn(self):
        input='7967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff'
        aux = TxIn.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TxOut(self):
        input='4baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac'
        aux = TxOut.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_TX(self):
        input='01000000017967a5185e907a25225574544c31f7b059c1a191d65b53dcc1554d339c4f9efc010000006a47304402206a2eb16b7b92051d0fa38c133e67684ed064effada1d7f925c842da401d4f22702201f196b10e6e4b4a9fff948e5c5d71ec5da53e90529c8dbd122bff2b1d21dc8a90121039b7bcd0824b9a9164f7ba098408e63e5b7e3cf90835cceb19868f54f8961a825ffffffff014baf2100000000001976a914db4d1141d0048b1ed15839d0b7a4c488cd368b0e88ac00000000'
        aux = TX.parse(input)
        output = aux.serialize().hex()
        self.assertEqual(input, output)

    def test_data_encoding(self):
        test = bytes(b'Hello World!')
        byte_count = len(test)
        encoded_data = data_encoding(test)
        self.assertEqual(byte_count, encoded_data[0])

    def test_data_decoding(self):
        test = bytes(b'Hello World!')
        encoded_data = data_encoding(test)
        encoded_data = encoded_data * 2
        decoded_data = data_decoding(BytesIO(encoded_data))
        self.assertEqual(test, decoded_data)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_TX (__main__.TestTX) ... ok
test_TxIn (__main__.TestTX) ... ok
test_TxOut (__main__.TestTX) ... ok
test_data_decoding (__main__.TestTX) ... ok
test_data_encoding (__main__.TestTX) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK
